In [2]:
# ! pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=f80c906ebdaa4d470f8a8801851ad2bff460848c121bfbd1cfee63edf66ff0eb
  Stored in directory: /home/seclab_taewan/.cache/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval
You should consider upgrading via the '/etc/anaconda3/envs/test/bin/python -m pip install --upgrade pip' command.


In [3]:
# ! pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 KB 8.9 MB/s eta 0:00:00
You should consider upgrading via the '/etc/anaconda3/envs/test/bin/python -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForTokenClassification, AdamW

In [5]:
# ! ls ../input

In [6]:
df_data = pd.read_csv("./tag_bio_char_.csv").fillna(method='ffill')
df_data.head()

,Sentence #,Word,Tag
0,Sentence: 1,경,O
1,Sentence: 1,력,O
2,Sentence: 2,증,O
3,Sentence: 2,명,O
4,Sentence: 2,서,O


In [7]:
df_data.Tag.value_counts()

I-BIR     1732648
I-MON     1326159
O          965540
I-PER      874896
B-PER      441439
B-BIR      208345
B-MON      168237
I-ADDR      35585
B-ADDR       8759
I-POL         240
B-POL          70
Name: Tag, dtype: int64

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
getter = SentenceGetter(df_data)
sentences = [[s[0] for s in sent] for sent in getter.sentences]
print(sentences[0])

['경', '력']


In [10]:
#poses = [[s[1] for s in sent] for sent in getter.sentences]
labels = [[s[1] for s in sent] for sent in getter.sentences]

print (sentences[0])
#print (poses[0])
print(labels[0])

['경', '력']
['O', 'O']


In [11]:
tags_vals = list(set(df_data["Tag"].values))

In [12]:
# Add X  label for word piece support
# Add [CLS] and [SEP] as BERT need
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)
tags_vals

{'B-ADDR',
 'B-BIR',
 'B-MON',
 'B-PER',
 'B-POL',
 'I-ADDR',
 'I-BIR',
 'I-MON',
 'I-PER',
 'I-POL',
 'O',
 'X',
 '[CLS]',
 '[SEP]'}

In [13]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
tag2idx={'B-PER': 1,
 'B-ADDR': 3,
 'B-BIR': 5,
 'B-MON': 7,
 'B-POL': 9,
 'I-PER': 2,
 'I-ADDR': 4,
 'I-BIR': 6,
 'I-MON': 8,
 'I-POL': 10,
 'X':11,
 'O': 0,
 '[CLS]':12,
 '[SEP]':13}
'''
tag2idx={'B-art': 14,
 'B-eve': 16,
 'B-geo': 0,
 'B-gpe': 13,
 'B-nat': 12,
 'B-org': 10,
 'B-per': 4,
 'B-tim': 2,
 'I-art': 5,
 'I-eve': 7,
 'I-geo': 15,
 'I-gpe': 8,
 'I-nat': 11,
 'I-org': 3,
 'I-per': 6,
 'I-tim': 1,
 'X':17,
 'O': 9,
 '[CLS]':18,
 '[SEP]':19}
 '''

"\ntag2idx={'B-art': 14,\n 'B-eve': 16,\n 'B-geo': 0,\n 'B-gpe': 13,\n 'B-nat': 12,\n 'B-org': 10,\n 'B-per': 4,\n 'B-tim': 2,\n 'I-art': 5,\n 'I-eve': 7,\n 'I-geo': 15,\n 'I-gpe': 8,\n 'I-nat': 11,\n 'I-org': 3,\n 'I-per': 6,\n 'I-tim': 1,\n 'X':17,\n 'O': 9,\n '[CLS]':18,\n '[SEP]':19}\n "

In [14]:
tag2idx

{'B-ADDR': 3,
 'B-BIR': 5,
 'B-MON': 7,
 'B-PER': 1,
 'B-POL': 9,
 'I-ADDR': 4,
 'I-BIR': 6,
 'I-MON': 8,
 'I-PER': 2,
 'I-POL': 10,
 'O': 0,
 'X': 11,
 '[CLS]': 12,
 '[SEP]': 13}

In [15]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [17]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 45
# load tokenizer, with manual file address or pretrained address
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 14476339.23B/s]


In [18]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:8
texts:[CLS] ᄀ ##ᅧ ##ᆼ ᄅ ##ᅧ ##ᆨ [SEP]
No.0,len:8
lables:[CLS] O X X O X X [SEP]
No.1,len:43
texts:[CLS] ᄀ ##ᅵ [UNK] ᄒ ##ᅡ ##ᆼ ᄀ ##ᅩ ##ᆼ ᄀ ##ᅩ ##ᆼ ᄒ ##ᅡ ##ᆨ ᄇ ##ᅮ ( ᄋ ##ᅮ ᄌ ##ᅮ ᄒ ##ᅡ ##ᆼ ᄀ ##ᅩ ##ᆼ ᄀ ##ᅩ ##ᆼ ᄒ ##ᅡ ##ᆨ ᄌ ##ᅥ ##ᆫ ᄀ ##ᅩ ##ᆼ ) [SEP]
No.1,len:43
lables:[CLS] O X O O X X O X X O X X O X X O X O O X O X O X X O X X O X X O X X O X X O X X O [SEP]
No.2,len:10
texts:[CLS] ᄀ ##ᅪ ##ᆼ ᄉ ##ᅡ ##ᆫ ᄀ ##ᅮ [SEP]
No.2,len:10
lables:[CLS] B-ADDR X X I-ADDR X X I-ADDR X [SEP]
No.3,len:10
texts:[CLS] ᄃ ##ᅡ ##ᆯ ᄌ ##ᅥ ##ᆫ ᄅ ##ᅵ [SEP]
No.3,len:10
lables:[CLS] B-ADDR X X I-ADDR X X I-ADDR X [SEP]
No.4,len:12
texts:[CLS] 8 0 8 6 1 0 0 0 0 0 [SEP]
No.4,len:12
lables:[CLS] B-MON I-MON I-MON I-MON I-MON I-MON I-MON I-MON I-MON I-MON [SEP]


In [19]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[  101  1455 30010 30025  1458 30010 30020   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


In [20]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])

[12  0 11 11  0 11 11 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [21]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [22]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [23]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [24]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [25]:
batch_num = 32

In [26]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [27]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased",num_labels=len(tag2idx))

100%|██████████| 440473133/440473133 [00:08<00:00, 50432282.05B/s]


In [28]:
model;

In [29]:
model.cuda();

RuntimeError: ignored

In [ ]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# Set epoch and grad max num
epochs = 6
max_grad_norm = 1.0

In [ ]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
# TRAIN loop
model.train();

In [36]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 739502
  Batch size = 32
  Num steps = 138660


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
Epoch:   0%|          | 0/6 [18:45<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
bert_out_address = 'models/bert_out_model/en09'
# Make dir if not exits
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [ ]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

In [ ]:
model = BertForTokenClassification.from_pretrained(bert_out_address,num_labels=len(tag2idx))

model.cuda(); # Set model to GPU

if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
model.eval();

In [ ]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

In [ ]:
import nltk
sentences = "I bought a new iphone and its Iphone X from Apple"
word_tokens = nltk.word_tokenize(sentences)
pos_tags = nltk.pos_tag(word_tokens)
tokenized_texts = []
word_piece_labels = []
i_inc = 0
temp_token = []
# Add [CLS] at the front 
temp_token.append('[CLS]')
for word,lab in pos_tags:
    token_list = tokenizer.tokenize(word)
    for m,token in enumerate(token_list):
        temp_token.append(token)
# Add [SEP] at the end
temp_token.append('[SEP]')
tokenized_texts.append(temp_token)
print("texts:%s"%(" ".join(temp_token)))
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])
b_input_mask = ""